In [1]:
#import necessary libraries
from mido import MidiFile, MidiTrack, Message
from mido.midifiles.meta import MetaMessage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import mido
import os
import math
import numpy as np
import pandas as pd
import math
from sklearn import metrics
import statistics
import random
import librosa
from music21 import converter, corpus, instrument, midi, note, chord, pitch, stream, tempo
import os
import itertools
import csv
import json

# Read tunes from Model Evaluation Dataset I

In [2]:
#read midis for port 
os.getcwd()
files_port = []
os.chdir("/home/girija/Documents/Project_Evaluation/Port/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_port.append(mido.MidiFile(file))

In [3]:
#get duration/ note-type
duration_port = []
note_port = []
for mid in files_port:
    d = []
    n = []
    delta = 0
    for track in mid.tracks:
        for message in track:
            if not isinstance(message, MetaMessage):
                d.append(message.time)
                n.append(message.type)
    duration_port.append(d[6:])
    note_port.append(n[6:])

In [4]:
#get pitch value, duration and note type
pitch_port = []
for mid in files_port:
    p = []
    for track in mid.tracks:
        for message in track:
            message = str(message)
            lines = message.splitlines()
            for line in lines:
                if('note_off' in line or 'note_on' in line):
                    props = line.split(' ')
                    p.append(int(props[3].split('=')[1]))
    pitch_port.append(p)

<h3>Get Binary Values for duration

In [5]:
l1 = []
for duration in duration_port:
    size = np.sum(np.asarray(duration))
    l1.append(size)
    
minimum_bin = min(l1)

In [6]:
port_binary = []
for item in duration_port:
    tmp = np.zeros(minimum_bin)
    i = 0
    for duration in item:
        if(duration==1 and i<minimum_bin):
                tmp[i] = 1
        else:
            tmp[i+1:duration+1] = 0
        i+=duration
    port_binary.append(list(tmp))

<h3>Get pitch values (compared with average)

In [7]:
#Find minimum length in jigs.
l = []
idx = []
for mid in pitch_port:
    l.append(len(mid))

#This will be the final minimum value to which the tunes will be cropped irrespective of their genre.
minimum_pitch = min(l)

In [8]:
#crop the notes to minimum length for both jigs and reels
input_data_port = []
for mid in pitch_port:
    input_data_port.append(mid[0:minimum_pitch])


In [9]:
#jigs
diff_notes2p = []
for item in input_data_port:
    avg = statistics.mean(item)
    diff = []
    for note in item:
        diff.append(note-avg)
    diff_notes2p.append(diff)

In [10]:
p = minimum_bin - minimum_pitch
padding = list(np.zeros(p))

In [11]:
for i in range(len(diff_notes2p)):
    diff_notes2p[i] = diff_notes2p[i]+padding

<h3>Get beats

In [12]:
#Read all jigs
os.getcwd()
files_port = []
os.chdir("/home/girija/Documents/Project_Evaluation/Port_wav/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_port.append(file)
        
beats_port = []
for file in files_port:
    y, sr = librosa.load(file)
    tempo, beat = librosa.beat.beat_track(y, sr)
    beats_port.append(beat)

In [13]:
#find minimum
#Find minimum length in jigs.
l = []
idx = []
for mid in beats_port:
    l.append(len(mid))

minimum_beats = min(l)

In [14]:
#crop to minimum
beats_p = []
for item in beats_port:
    beats_p.append(list(item[:minimum_beats]))

In [15]:
#padding
p = minimum_bin - minimum_beats
padding = list(np.zeros(p))
for i in range(len(beats_p)):
    beats_p[i] = beats_p[i]+padding

In [16]:
#create Datapoints:
datasets_port = []
for i in range(len(port_binary)):
    datasets_port.append(np.asarray([port_binary[i],diff_notes2p[i],beats_p[i]]))

<h3>K-Means Clustering

In [17]:
def KMeansClustering(dataset, k, max_itr):
    #create an empty list of centroids
    centroids = []
    #select first 'k' datapoints as the centroids
    for i in range(k):
        centroids.append(dataset[i])
    
    for itr in range(max_itr):
        clusters = []
        #find distance between a data-point and each centroid
        for point in dataset:
            distances = [np.linalg.norm(point-centroid) for centroid in centroids]
            #select centroid at minimum distance
            clusters.append(distances.index(min(distances)))

        #empty dictionary to store data-points as per cluster numbers    
        data = {}
        for i in range(k):
            data[i] = []
        
        #append data-points to the list based on cluster number    
        for i in range(len(dataset)):
            data[clusters[i]].append(dataset[i])
        
        #update centroids based on mean of data-points in the cluster    
        centroids = []
        for point in data:
            centroids.append(np.mean(data[point],axis = 0))
            
    return(clusters)

In [18]:
clusters_port = KMeansClustering(datasets_port, 7, 100)

# Read tunes from Model Evaluation Dataset II

In [19]:
#read midis for port 
os.getcwd()
files_session = []
os.chdir("/home/girija/Documents/Project_Evaluation/session/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_session.append(mido.MidiFile(file))

In [20]:
#get duration/ note-type
pitch_session = []
duration_session = []
note_session = []
for mid in files_session:
    p = []
    d = []
    n = []
    delta = 0
    for track in mid.tracks:
        for message in track:
            if not isinstance(message, MetaMessage):
                p.append(message.note)
                d.append(message.time)
                n.append(message.type)
    duration_session.append(d)
    note_session.append(n)
    pitch_session.append(p)

<h3>Get Binary Values for duration

In [21]:
l1 = []
for duration in duration_session:
    size = np.sum(np.asarray(duration))
    l1.append(size)
    
minimum_bin = min(l1)

In [22]:
session_binary = []
for item in duration_session:
    tmp = np.zeros(minimum_bin)
    i = 0
    for duration in item:
        if(duration==1 and i<minimum_bin):
                tmp[i] = 1
        else:
            tmp[i+1:duration+1] = 0
        i+=duration
    session_binary.append(list(tmp))

<h3>Get pitch values (compared with average)

In [23]:
#Find minimum length in jigs.
l = []
idx = []
for mid in pitch_session:
    l.append(len(mid))

#This will be the final minimum value to which the tunes will be cropped irrespective of their genre.
minimum_pitch = min(l)

In [24]:
#crop the notes to minimum length for both jigs and reels
input_data_session = []
for mid in pitch_session:
    input_data_session.append(mid[0:minimum_pitch])

In [25]:
#jigs
diff_notes2s = []
for item in input_data_session:
    avg = statistics.mean(item)
    diff = []
    for note in item:
        diff.append(note-avg)
    diff_notes2s.append(diff)

In [26]:
p = minimum_bin - minimum_pitch
padding = list(np.zeros(p))

In [27]:
for i in range(len(diff_notes2s)):
    diff_notes2s[i] = diff_notes2s[i]+padding

<h3>Get beats

In [28]:
#Read all jigs
os.getcwd()
files_session = []
os.chdir("/home/girija/Documents/Project_Evaluation/session_wav/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_session.append(file)
        
beats_session = []
for file in files_session:
    y, sr = librosa.load(file)
    tempo, beat = librosa.beat.beat_track(y, sr)
    beats_session.append(beat)

In [29]:
#find minimum
#Find minimum length in jigs.
l = []
idx = []
for mid in beats_session:
    l.append(len(mid))

minimum_beats = min(l)

In [30]:
#crop to minimum
beats_s = []
for item in beats_session:
    beats_s.append(list(item[:minimum_beats]))

In [31]:
#padding
p = minimum_bin - minimum_beats
padding = list(np.zeros(p))
for i in range(len(beats_s)):
    beats_s[i] = beats_s[i]+padding

In [32]:
#create Datapoints:
datasets_session = []
for i in range(len(session_binary)):
    datasets_session.append(np.asarray([session_binary[i],diff_notes2s[i],beats_s[i]]))

In [33]:
clusters_session = KMeansClustering(datasets_session, 7, 100)

<h3>Similarity Function for port: comparing port tunes to port tunes

In [34]:
#dictionary of tune number and track name
d = {
    0:'A fig for a kiss',
    1:'An Buachaill Caol Dubh',
    2:'An fhinne-bhean mhodhamhuil',
    3:'Bean dubh an ghleanna',
    4:'Bonnie Kate',
    5:'Bruach na carraige b�ine',
    6:'Easter snow',
    7:'Humours of Ennistymon',
    8:'Hunt the cat',
    9:'My love is in America',
    10:'Rakish Paddy',
    11:'Sliabh na mBan',
    12:'The ace and deuce of pipering',
    13:'The blackberry blossom',
    14:'The blackbird',
    15:'The boys of the lough',
    16:'The bucks of Oranmore',
    17:'The dusty miller',
    18:'The flogging reel',
    19:'The garden of daisies',
    20:'The geese in the bog',
    21:'The heather breeze',
    22:'The lark in the morning',
    23:'The Leitrim fancy',
    24:'The maid on the green',
    25:'The morning star',
    26:'The pigeon on the gate',
    27:'The star of Munster',
    28:'The Wild Irishman',
    29:'The yellow wattle',
    30:'Toss the feathers',
    31:'Touch me if you dare'
}

In [35]:
similar2 = []
#for every tune, find top ten similar tunes
for i in range(len(datasets_port)):
    dist = []
    for j in range(len(datasets_port)):
        dist.append(np.linalg.norm(datasets_port[i]-datasets_port[j]))
    sortedlist = list(np.sort(dist))[1:11]
    
    indices = []
    for item in sortedlist:
        indices.append(dist.index(item))
    similar2.append(indices)

In [36]:
os.getcwd()
os.chdir("/home/girija/Documents/Project_Evaluation/")

import csv
with open('tuneAndGenre.csv') as f:
    genre = dict(filter(None, csv.reader(f)))

In [37]:
#create CSV
os.getcwd()
os.chdir("/home/girija/Documents/Project_Evaluation/")
i = 0
with open('topten_usingSim2.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['TrackName','Genre',
                         'Tune1','Genre1', 'Tune2','Genre2','Tune3','Genre3',
                         'Tune4','Genre4','Tune5','Genre5',
                         'Tune6','Genre6','Tune7','Genre7','Tune8','Genre8',
                         'Tune9','Genre9','Tune10','Genre10'
                        ])
    for tunes_list in similar2:
        row = [d[i],genre[d[i]]]
        for tune in tunes_list:
            row.append(d[tune])
            row.append(genre[d[tune]])
        filewriter.writerow(row)
        i+=1

<h3>Similarity Function for session: comparing session tunes to session tunes

In [38]:
#for every tune, find top ten similar tunes
similar2_session = []
for i in range(len(datasets_session)):
    dist = []
    for j in range(len(datasets_session)):
        dist.append(np.linalg.norm(datasets_session[i]-datasets_session[j]))
    sortedlist = list(np.sort(dist))[1:11]
    
    indices = []
    for item in sortedlist:
        indices.append(dist.index(item))
    similar2_session.append(indices)

In [39]:
#create CSV
os.getcwd()
os.chdir("/home/girija/Documents/Project_Evaluation/")
i = 0
with open('topten_usingSim2_session.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['TrackName','Genre',
                         'Tune1','Genre1', 'Tune2','Genre2','Tune3','Genre3',
                         'Tune4','Genre4','Tune5','Genre5',
                         'Tune6','Genre6','Tune7','Genre7','Tune8','Genre8',
                         'Tune9','Genre9','Tune10','Genre10'
                        ])
    for tunes_list in similar2_session:
        row = [d[i],genre[d[i]]]
        for tune in tunes_list:
            row.append(d[tune])
            row.append(genre[d[tune]])
        filewriter.writerow(row)
        i+=1